# Deploy model

In [1]:
%reload_ext autoreload
%autoreload 2

## Setup

In [2]:
import os
import shutil
from fastai.vision import *
import sys
sys.path.append('../scripts/')
import mfc_functions
import IPython.display as ipd

In [3]:
kMY_WAVS_DIR = './my_wavs'
kMY_MFCS_DIR = './my_mfcs'
path_img = '../mfc_dataset'  

# export.pkl is stored together with the training + validation data inside path_img.
# export.pkl is generated in `03_train_model.ipynb` by calling learn.export().

In case you want to delete the MFC JPEGs generated by me:

In [4]:
# shutil.rmtree(kMY_MFCS_DIR)
# os.makedirs(kMY_MFCS_DIR)

## Convert WAVs in `./my_wavs` to MFC JPEGs in `./my_mfcs`

In [5]:
mfc_functions.kDEBUG = False  # print out the length and the array for each WAV for error spotting

In [6]:
def is_wav(fname):
    return str(fname).split('.')[-1] == 'wav'

In [7]:
for fname in os.listdir(kMY_WAVS_DIR):
    if is_wav(fname):
        wav_fname = fname
        wav_fpath = os.path.join(kMY_WAVS_DIR, fname)
        mfc_functions.pipeline(
            wav_fpath=wav_fpath, 
            save_fpath=os.path.join(kMY_MFCS_DIR, wav_fname.split('.')[0] + '.jpg')
        )

## Instantiate a Inference Learner

In [8]:
learn = load_learner(path_img, test=ImageList.from_folder(kMY_MFCS_DIR))

In [9]:
learn.data.test_ds.items

array([PosixPath('my_mfcs/me_eight_male.jpg'), PosixPath('my_mfcs/me_six_male.jpg'),
       PosixPath('my_mfcs/me_seven_male.jpg'), PosixPath('my_mfcs/me_complicated_male.jpg'),
       PosixPath('my_mfcs/mywife_six_maleLike.jpg'), PosixPath('my_mfcs/me_observations_male.jpg'),
       PosixPath('my_mfcs/me_transparency_male.jpg'), PosixPath('my_mfcs/me_two_male.jpg'),
       PosixPath('my_mfcs/me_nine_male.jpg'), PosixPath('my_mfcs/me_one_male.jpg'),
       PosixPath('my_mfcs/me_four_male.jpg'), PosixPath('my_mfcs/me_three_male.jpg'),
       PosixPath('my_mfcs/me_five_male.jpg'), PosixPath('my_mfcs/me_what_male.jpg'),
       PosixPath('my_mfcs/mywife_one_maleLike.jpg'), PosixPath('my_mfcs/me_nine_femaleLike.jpg')], dtype=object)

## Inference

In [10]:
preds, _ = learn.get_preds(ds_type=DatasetType.Test)

In [11]:
i2c = { v : k for k, v in learn.data.c2i.items() }

Surprisingly, when I inputted audios that are not digits (e.g. "transparency"), the model still classified them correctly. 

My wife had one successful attempt at fooling the model to classify her voice as "male".

I had one successful attempt at fooling the model to classify my voice as female".

In [12]:
print('{:40s} {:15s}'.format('File Path', 'Classification'))
print(''.join(['='] * 55))

for mfc_fpath, y in zip(learn.data.test_ds.items, preds.argmax(axis=1).numpy()):
    print('{:40s} {:15s}'.format(str(mfc_fpath), i2c[y]))

File Path                                Classification 
my_mfcs/me_eight_male.jpg                male           
my_mfcs/me_six_male.jpg                  male           
my_mfcs/me_seven_male.jpg                male           
my_mfcs/me_complicated_male.jpg          male           
my_mfcs/mywife_six_maleLike.jpg          male           
my_mfcs/me_observations_male.jpg         male           
my_mfcs/me_transparency_male.jpg         male           
my_mfcs/me_two_male.jpg                  male           
my_mfcs/me_nine_male.jpg                 male           
my_mfcs/me_one_male.jpg                  male           
my_mfcs/me_four_male.jpg                 male           
my_mfcs/me_three_male.jpg                male           
my_mfcs/me_five_male.jpg                 male           
my_mfcs/me_what_male.jpg                 male           
my_mfcs/mywife_one_maleLike.jpg          female         
my_mfcs/me_nine_femaleLike.jpg           female         


For each classification, you can listen to its corresponding WAV below to verify:

In [13]:
for fname in os.listdir(kMY_WAVS_DIR):
    if is_wav(fname):
        wav_fname = fname
        wav_fpath = os.path.join(kMY_WAVS_DIR, wav_fname)
        print(wav_fpath)
        ipd.display(ipd.Audio(wav_fpath))

./my_wavs/me_five_male.wav


./my_wavs/mywife_six_maleLike.wav
./my_wavs/me_six_male.wav


./my_wavs/me_nine_femaleLike.wav


./my_wavs/me_three_male.wav


./my_wavs/me_two_male.wav


...

./my_wavs/me_transparency_male.wav


./my_wavs/me_one_male.wav


./my_wavs/me_what_male.wav


./my_wavs/me_complicated_male.wav


./my_wavs/me_nine_male.wav
